In [2]:
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd

Cars_page_url = "https://bikroy.com/en/ads/bangladesh/cars"
Cars_page_from2nd = "https://bikroy.com/en/ads/bangladesh/cars?sort=date&order=desc&buy_now=0&urgent=0&page=" # add 2,3... at the end of the link for iterating though pages

## per page car = 25
## available pages = 300+
## cars class has this "gtm-normal-ad"
# 
# Collect:
# - brand
# - model
# - year
# - condition
# - transmission
# - fuel_type
# - kilometers_run
# - price (from the listing)







In [ ]:
carDataDict = {
    'cars':[],
    'price': [],
    'brand': [],
    'model': [],
    'year': [],
    'condition': [],
    'transmission': [],
    'fuel_type': [],
    'kilometers_run': []
}


def extract_value(detailsDiv, key):
    labels = detailsDiv.find_all('div', class_='label--3oVZK')
    values = detailsDiv.find_all('div', class_='value--1lKHt')
    
    for label, value in zip(labels, values):
        label_text = label.text.strip().lower().replace(":", "")
        if key.lower() in label_text:
            return value.get_text(strip=True)
    return "NaN"








def clearDict(dict):
    for key in dict:
        dict[key]=[]
        
clearDict(carDataDict)


    

,cars,price,brand,model,year,condition,transmission,fuel_type,kilometers_run


In [87]:
BASE_URL = "https://bikroy.com"
def getCarData(relative_url):
    car_url = BASE_URL+relative_url
    if(car_url in carDataDict['cars']):
        return
    r = requests.get(car_url)
    carPage = bs(r.content)
    
    #get details div
    detailsDiv = carPage.find_all('div',attrs={'class':'section--PpGYD'})
    if(detailsDiv):
        detailsDiv = detailsDiv[0]
        # print(detailsDiv.prettify())
    else:
        return
        # print("No Details field")
        
    # Get price
    price = detailsDiv.find_all('div',attrs={'class':'amount--3NTpl'})
    if(price):
        price = price[0]
        price = price.text.strip()
        carDataDict['price'].append(price)
    else:
        carDataDict['price'].append("NaN")
    
        
    # Get brand
    brand = extract_value(detailsDiv,'Brand')
    carDataDict['brand'].append(brand)

    # Get model
    model = extract_value(detailsDiv,'model')
    carDataDict['model'].append(model)
    
    # Get year of manufacture
    year = extract_value(detailsDiv,'year of manufacture')
    carDataDict['year'].append(year)
    # Get condition
    condition = extract_value(detailsDiv,'condition')
    carDataDict['condition'].append(condition)
    # Get transmission type
    Transmission = extract_value(detailsDiv,'Transmission')
    carDataDict['transmission'].append(Transmission)
    # Get fuel type
    fuel = extract_value(detailsDiv,'fuel')
    carDataDict['fuel_type'].append(fuel)
    # Get kilometers run
    km_run = extract_value(detailsDiv,'kilometers')
    carDataDict['kilometers_run'].append(km_run)
    
    
    
    carDataDict['cars'].append(car_url)


    

In [ ]:
## function for collecting data of one page
def getPageData(page_url):
    r = requests.get(page_url)
    page = bs(r.content)
    all_li = page.find_all('li',attrs={'class':'gtm-top-ad'})
    for li in all_li:
        a = li.find_all('a')[0]
        link = a.get('href')
        getCarData(link)

def getData(url):
    page_count = 2
    for page_count in range(200):
        getPageData(url+str(page_count))

getData("https://bikroy.com/en/ads/bangladesh/cars?sort=date&order=desc&buy_now=0&urgent=0&page=")
    

    
    
    
    # """
    # 1. parse page html
    # 2. get all li
    # 3. get all li that has "gtm-normal-ad" in class
    # 4. pass each car link to getCarData()
    # 5. 
    # """

In [ ]:
carDataDF = pd.DataFrame(carDataDict)
carDataDF.to_csv("Data/raw_data.csv",index=False)


,cars,price,brand,model,year,condition,transmission,fuel_type,kilometers_run
0,https://bikroy.com/en/ad/toyota-noah-x-smart-n...,"Tk 3,900,000",Toyota,Noah,2020,Reconditioned,Automatic,Octane,"100,000 km"
1,https://bikroy.com/en/ad/toyota-prius-phv-a-pr...,"Tk 3,390,000",Toyota,Prius,2020,Reconditioned,Automatic,"Hybrid, Electric, Octane","26,000 km"
2,https://bikroy.com/en/ad/toyota-allion-2002-fo...,"Tk 1,110,000",Toyota,Allion,2002,Used,Automatic,"CNG, Octane","11,000 km"
3,https://bikroy.com/en/ad/toyota-noah-full-load...,"Tk 3,650,000",Toyota,Noah,2020,Reconditioned,Automatic,"Hybrid, Octane","92,153 km"
4,https://bikroy.com/en/ad/mercedes-benz-g-class...,"Tk 92,000,000",Mercedes-Benz,G-Class,2022,Reconditioned,Automatic,Petrol,"6,000 km"
...,...,...,...,...,...,...,...,...,...
95,https://bikroy.com/en/ad/toyota-axio-ex-pkg-hv...,"Tk 2,360,000",Toyota,Axio,2020,Reconditioned,Automatic,"Petrol, Hybrid","79,000 km"
96,https://bikroy.com/en/ad/toyota-prado-tx-ltd-2...,"Tk 7,300,000",Toyota,Prado,2012,Used,Automatic,Octane,"43,300 km"
97,https://bikroy.com/en/ad/nissan-bluebird-2008-...,"Tk 1,050,000",Nissan,Bluebird,2008,Used,Automatic,"Petrol, CNG","291,856 km"
98,https://bikroy.com/en/ad/toyota-corolla-g-2022...,"Tk 4,390,000",Toyota,Corolla,2022,Used,Automatic,Octane,"49,000 km"
